In [ ]:
import numpy
import numpy as np
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
import os
import pickle
from scipy.sparse.linalg import eigsh
from scipy import sparse
from numpy import genfromtxt

In [ ]:
!pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 51.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
# initialize variables
# parameters
norb = 10
N = norb
na_true = 5
nb_true = na_true

In [ ]:
import multiprocessing
# Get the number of CPU cores
num_workers = multiprocessing.cpu_count()
print(f"Number of available workers (CPU cores): {num_workers}")

Number of available workers (CPU cores): 8


In [ ]:
from pyscf.fci import cistring
alpha_list = list(cistring.gen_strings4orblist(range(norb), na_true))
beta_list  = list(cistring.gen_strings4orblist(range(norb), nb_true))

# Build a list of (alpha, beta) pairs in EXACT the same order as PySCF:
basis_pair = []
for a in alpha_list:
    for b in beta_list:
        basis_pair.append((a, b))

pair_to_index = { ab_pair: i for i, ab_pair in enumerate(basis_pair) }
# Now pair_to_index[(alpha_int, beta_int)] = index in PySCF's ordering

def D2B(num, W):
    # Build a bit array so that array[0] = LSB = orbital 0, array[W-1] = MSB = orbital W-1
    # E.g. num=14 -> bin(num)='1110' -> reversed bits => array=[0,1,1,1]
    bits_str = bin(num)[2:].rjust(W, '0')  # e.g. "1110"
    # Reverse so that array[0] is the last digit from bits_str
    return np.array(list(bits_str[::-1]), dtype=int)

def B2D(array):
    # Now array[0] is the LSB, array[-1] is the MSB
    res = 0
    for i, bit in enumerate(array):
        if bit == 1:
            res |= (1 << i)
    return res

In [ ]:
print('basis dimension is',len(basis_pair))
# print(basis_pair[:10])

basis dimension is 63504


In [ ]:
# get the potential list for each number operator in product state basis bascailly to check if there's one or 2 eletrons
number_op_list = []
# the U part
for i in range(N): # this is for U term, loop N which is each site
    number_occ_list =[]
    for index, pair in enumerate(basis_pair):
        alpha_array = D2B(pair[0], norb)
        beta_array  = D2B(pair[1], norb)
         # i is the site index
        number_occ = np.int8(alpha_array[i]*beta_array[i])
        number_occ_list.append(number_occ) # number of doulbe occ
    number_op_list.append(number_occ_list)
###########################################################################################

# # coulumb interaction but different orbitals , alpha and beta
# for i in range(N):
#     for j in range(N):
#         if j>i:
#             number_occ_list =[]
#             for index, pair in enumerate(basis_pair):
#                 alpha_array = D2B(pair[0], norb)
#                 beta_array  = D2B(pair[1], norb)
#                 number_occ = np.int8( alpha_array[i]*beta_array[j] + beta_array[i]*alpha_array[j] )
#                 number_occ_list.append(number_occ)
#             number_op_list.append(number_occ_list)

# ###########################################################################################

# # exchange interaction different orbitals , same spin
# for i in range(N):
#     for j in range(N):
#         if j>i:
#             number_occ_list =[]
#             for index, pair in enumerate(basis_pair):
#                 alpha_array = D2B(pair[0], norb)
#                 beta_array  = D2B(pair[1], norb)
#                 number_occ = np.int8( alpha_array[i]*alpha_array[j] + beta_array[i]*beta_array[j] )
#                 number_occ_list.append(number_occ)
#             number_op_list.append(number_occ_list)

In [ ]:
number_op_list = np.array(number_op_list)


string_name = f'number_op_list_norb={N}_na_{na_true}_nb_{nb_true}.txt'
if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/{string_name}'
else:
    file_path = string_name

np.savetxt(file_path, number_op_list,fmt='%d')

In [ ]:
# Initialize the dictionary for hopping pairs
nn_condition = False  # True for nearest-neighbor hopping
hopping_cos_dict = {}
for j in range(N):
    for k in range(N):
        if nn_condition:
            if j == k+1:
                hopping_cos_dict[(j, k, 1)] = []  # Key for positive spin
                hopping_cos_dict[(j, k, -1)] = []  # Key for negative spin
        else:
            if k<j:
                # Include keys for both positive and negative spins
                hopping_cos_dict[(j, k, 1)] = []  # Key for positive spin
                hopping_cos_dict[(j, k, -1)] = []  # Key for negative spin

# Populate the dictionary
for (j, k, spin) in hopping_cos_dict.keys():
    good_index_list = []
    for index, pair in enumerate(basis_pair):
        alpha_array = D2B(pair[0], norb)
        beta_array  = D2B(pair[1], norb)
        if spin == 1:
            value = alpha_array[j] + alpha_array[k]
        else:
            value = beta_array[j] + beta_array[k]
        # here i dont care if it is i to j or j to i, bc it will be hermitian when implemented later in teh UCC_t_part code
        if value == 1:
            good_index_list.append(np.int64(index))
            # THIS INT64 CAN NOT BE CHANGED, IT WILL BE USED AS INDEX IN THE FUTURE
            ################################################################################
            ################################################################################
    # Update the dictionary
    hopping_cos_dict[(j, k, spin)] = np.array(good_index_list)

# Now hopping_cos_dict contains the good index lists for all (i, j, spin) pairs

In [ ]:
string_name = f'torch_hopping_cose_dict_norb={N}_na_{na_true}_nb_{nb_true}.pkl'
if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/{string_name}'
else:
    file_path = string_name

with open(file_path, 'wb') as f:
    pickle.dump(hopping_cos_dict, f)

In [ ]:
# Initialize the dictionary for hopping pairs
hopping_dict = {}
for j in range(N):
    for k in range(N):
        if nn_condition:
            if j == k+1:
                hopping_dict[(j, k, 1)] = {'row': [], 'col': [], 'data': []}
                hopping_dict[(j, k, -1)] = {'row': [], 'col': [], 'data': []}
        else:
            if k<j:
                hopping_dict[(j, k, 1)] = {'row': [], 'col': [], 'data': []}  # Key for positive spin
                hopping_dict[(j, k, -1)] = {'row': [], 'col': [], 'data': []}  # Key for negative spin

In [ ]:
#############################
# 2) FUNCTION FOR ONE INDEX #
#############################

def process_single_index(index):
    """
    Compute the hopping contributions for a single basis-pair index.
    Returns a dictionary with keys (j_loc, k_loc, spin_sign),
    each value is a dict containing 'row', 'col', 'data' lists.
    """
    local_dict = {}

    pair = basis_pair[index]      # pair is (alphaDetInt, betaDetInt)
    alpha_array = D2B(pair[0], norb)
    beta_array  = D2B(pair[1], norb)

    for j_loc in range(N):       # "destruction" index
        for k_loc in range(N):   # "creation" index
            # Condition checks
            if nn_condition:
                # Only allow j_loc == k_loc + 1
                if j_loc != k_loc + 1:
                    continue
            else:
                # Only allow k_loc < j_loc
                if k_loc >= j_loc:
                    continue

            # For each spin sign
            for spin_sgn in [1, -1]:
                state_array = alpha_array if spin_sgn == 1 else beta_array

                # Check if we can "hop" from j_loc to k_loc
                if state_array[j_loc] == 1 and state_array[k_loc] == 0:
                    old_det_int  = B2D(state_array)
                    phase_factor = cistring.cre_des_sign(k_loc, j_loc, old_det_int)

                    # Hop the electron
                    new_state_array = state_array.copy()
                    new_state_array[j_loc] = 0
                    new_state_array[k_loc] = 1

                    # Build the new tuple
                    if spin_sgn == 1:
                        new_tuple = (B2D(new_state_array), B2D(beta_array))
                    else:
                        new_tuple = (B2D(alpha_array), B2D(new_state_array))

                    # Get index of new_tuple in basis_pair
                    # If your basis_pair is huge, consider a dict mapping {tuple: idx} for speed
                    col_index = basis_pair.index(new_tuple)

                    key = (j_loc, k_loc, spin_sgn)
                    if key not in local_dict:
                        local_dict[key] = {'row': [], 'col': [], 'data': []}
                    # need to change int data type if working with more than 16 spatial orbitals
                    ################################################################################
                    ################################################################################
                    local_dict[key]['row'].append(np.int64(index))
                    local_dict[key]['col'].append(np.int64(col_index))
                    local_dict[key]['data'].append(np.int8(phase_factor))
                    ################################################################################
    return local_dict

######################################
# 3) CHUNKING PARAMETERS AND FOLDERS #
######################################

num_cores = 16               # Number of CPU cores to use
total_dets = len(basis_pair)   # Total number of basis-pair states
chunk_size = 50000            # Adjust based on memory/time constraints
num_chunks = (total_dets + chunk_size - 1) // chunk_size


if IN_COLAB:
    save_folder = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/op_pieces'
else:
    save_folder = os.path.join(os.getcwd(), "op_pieces")
os.makedirs(save_folder, exist_ok=True)

##########################
# 4) PROCESS EACH CHUNK  #
##########################

def process_chunk(chunk_idx):
    """
    Process the chunk of indices for chunk_idx,
    and save the partial hopping dictionary to a pickle file.
    """
    import pickle  # local import to avoid issues in parallel
    from tqdm import tqdm  # local import for progress bar

    chunk_file = os.path.join(save_folder, f"hopping_chunk_{chunk_idx}.pkl")
    if os.path.exists(chunk_file):
        print(f"[Chunk {chunk_idx}] Already exists. Skipping.")
        return  # Skip re-computation

    start_idx = chunk_idx * chunk_size
    end_idx = min((chunk_idx + 1) * chunk_size, total_dets)
    print(f"[Chunk {chunk_idx}] Processing indices {start_idx} to {end_idx}...")

    # Parallel compute local dictionaries for each index in [start_idx, end_idx)
    # We attach a progress bar *outside* the parallel function to reduce pickling issues.
    results = []
    for i in tqdm(range(start_idx, end_idx), desc=f"Chunk {chunk_idx}"):
        # Just call process_single_index directly in a loop
        # If you want parallel calls, see below for how to do it carefully
        results.append(process_single_index(i))

    # If you prefer parallel calls, you can do:
    # results = Parallel(n_jobs=num_cores)(
    #     delayed(process_single_index)(i) for i in range(start_idx, end_idx)
    # )
    # but you might need to remove the tqdm or wrap it carefully.

    # Merge these local dicts into one chunk dict
    chunk_hopping_dict = {}
    for local_res in results:
        for key, val in local_res.items():
            if key not in chunk_hopping_dict:
                chunk_hopping_dict[key] = {'row': [], 'col': [], 'data': []}
            chunk_hopping_dict[key]['row'].extend(val['row'])
            chunk_hopping_dict[key]['col'].extend(val['col'])
            chunk_hopping_dict[key]['data'].extend(val['data'])

    # Save the chunk dictionary to disk
    with open(chunk_file, 'wb') as f:
        pickle.dump(chunk_hopping_dict, f)

    print(f"[Chunk {chunk_idx}] Saved at {chunk_file}")

###################
# 5) RUN ALL CHUNKS
###################
for chunk_idx in range(num_chunks):
    process_chunk(chunk_idx)

##############################
# 6) CHECK WHICH CHUNKS EXIST
##############################
chunk_files = []
for i in range(num_chunks):
    fname = os.path.join(save_folder, f"hopping_chunk_{i}.pkl")
    if os.path.exists(fname):
        chunk_files.append(fname)

print(f"Found {len(chunk_files)}/{num_chunks} chunk files.")

#########################################
# 7) MERGE CHUNKS IF ALL ARE AVAILABLE  #
#########################################
if len(chunk_files) == num_chunks:
    print("All chunk files present. Merging into final hopping_dict...")

    hopping_dict = {}
    for cf in tqdm(chunk_files, desc="Merging chunks"):
        with open(cf, 'rb') as f:
            partial_dict = pickle.load(f)

        # Merge partial_dict into global hopping_dict
        for key, val in partial_dict.items():
            if key not in hopping_dict:
                hopping_dict[key] = {'row': [], 'col': [], 'data': []}
            hopping_dict[key]['row'].extend(val['row'])
            hopping_dict[key]['col'].extend(val['col'])
            hopping_dict[key]['data'].extend(val['data'])

    # Convert each entry in hopping_dict to a sparse CSR matrix
    for key in hopping_dict:
        row = np.array(hopping_dict[key]['row'])
        col = np.array(hopping_dict[key]['col'])
        data = np.array(hopping_dict[key]['data'])
        hopping_dict[key] = sparse.csr_matrix(
            (data, (row, col)), shape=(total_dets, total_dets)
        )

    # Save final dictionary
    final_file = os.path.join(save_folder, f"torch_sparse_matrices_dict_norb={norb}_na_{na_true}_nb_{nb_true}.pkl")
    with open(final_file, 'wb') as f:
        pickle.dump(hopping_dict, f)

    print(f"Final hopping_dict saved at: {final_file}")

    # Optionally remove the chunk files
    # for cf in chunk_files:
    #     os.remove(cf)
    # print("All chunk files removed after merging.")

else:
    print("Not all chunks are present. Rerun or check partial runs.")

[Chunk 0] Processing indices 0 to 50000...


Chunk 0: 100%|██████████| 50000/50000 [18:37<00:00, 44.73it/s]


[Chunk 0] Saved at /content/drive/My Drive/Quantum_chemistry/hardware_effient/op_pieces/hopping_chunk_0.pkl
[Chunk 1] Processing indices 50000 to 63504...


Chunk 1: 100%|██████████| 13504/13504 [14:01<00:00, 16.05it/s]


[Chunk 1] Saved at /content/drive/My Drive/Quantum_chemistry/hardware_effient/op_pieces/hopping_chunk_1.pkl
Found 2/2 chunk files.
All chunk files present. Merging into final hopping_dict...


Merging chunks: 100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


Final hopping_dict saved at: /content/drive/My Drive/Quantum_chemistry/hardware_effient/op_pieces/torch_sparse_matrices_dict_norb=10_na_5_nb_5.pkl
